<a href="https://colab.research.google.com/github/papajo/2015/blob/master/Custom_Component.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add a Custom Component
We offer support on adding your own components apart from the supported ones (generation, fusion, ranking, etc). This allows for you to add your own custom components

Here you will see how to create a component, add it to your configs, and add it to Archon with add_component.

Furthermore, you can pass a custom state between components, allowing you to have multiple custom components that can send information between eachother.

Alternatively, you can also edit the codebase and add it as a component to `archon/components/`. If doing this route, make sure to also add your component to `__init__.py` within the above directory so it can be imported.



In [ ]:
!pip install archon-ai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
from archon.completions import Archon # 'Archon' is the initialization class
from archon.completions.components import Component, Generator #

import os
os.environ['TOGETHER_API_KEY'] = "<your-together-key-here"

In [ ]:
# Component superclass requires you to have __init__ and generate to
# interact with the Archon architecture.
class custom_component(Component):
    def __init__(self, config):
        self.config = config

        self.model_name = self.config["model"]
        self.model_type = self.config["model_type"]
        self.temperature = self.config["temperature"]
        self.max_tokens = self.config["max_tokens"]
        self.samples = self.config["samples"]

        # Generator class helps us handle messaging
        self.model = Generator(self.config)

    def run(self, conversation: list, prev_state, state):
        """
        Run the component and updates the state accordingly.

        Args:
            conversation (list[dict]): A list of dictionaries representing the conversation with Archon.
                Each dictionary contains role and content
            prev_state (dict): A dictionary representing the state from the previous layer.
            state (dict): A dictionary holding the values that will be updated from the previous layer to be sent to the next layer
        """

        candidates = prev_state["candidates"]

        fused_generations = self.fuse_singer(conversation, candidates)

        state["candidates"].extend(fused_generations)

        return



    def fuse_singer(self, conversation: list, candidates: list):

        # For this example, our component will fuse and turn the output into a song
        query = conversation[-1]["content"]

        prompt = f"You have been provided with a set of responses from various models to the this query: {query}. \
            Your task is to synthesize these responses into a memerable song that best answers the query"
        for i, reference in enumerate(candidates):
            prompt += f"\n{i+1}. {reference}"

        messages = (
            [
                {
                    "role": "system",
                    "content": "You are a helpful assistant who fuses answers and turns them into songs",
                }
            ]  # system
            + [
                message for message in conversation[:-1] if message["role"] != "system"
            ]  # rest of conversation without query
            + [{"role": "user", "content": prompt}]  # prompt
        )

        fuser_generations = []
        for _ in range(self.samples):
            output = self.model.generate_from_messages(
                messages, temperature=self.temperature
            )
            if output is not None:
                fuser_generations.extend(output)

        return fuser_generations


In [ ]:
# Now that you've made your component, you can add it to your config
archon_config = {
    "name": "archon-testing",
    "custom": True,  # SET THE CONFIG TO USE CUSTOM
    "layers": [
        [
            {
                "type": "generator",
                "model": "Qwen/Qwen2-72B-Instruct",
                "model_type": "Together_API",
                "temperature": 0.7,
                "max_tokens": 2048,
                "samples": 1,
            }
        ],
        [
            {
                "type": "custom_singer",  # custom type here
                "model": "Qwen/Qwen2-72B-Instruct",
                "model_type": "Together_API",
                "temperature": 0.7,
                "max_tokens": 2048,
                "samples": 1,
            }
        ],
    ],
}

In [ ]:
# Start with custom config
archon = Archon(archon_config)

# Add component to Archon
# name has to match config
archon.add_component("custom_singer", custom_component)

# Have to manually initialize
archon.initialize()

# make conversation
testing_instruction = [{"role": "user", "content": "How do I make a cake?"}]

response = archon.generate(testing_instruction)

print(response)

2024-10-22 04:41:06.297 | WARNING  | archon.completions.archon:__init__:151 - Custom model, make sure to add custom components before initializing.
2024-10-22 04:41:06.304 | INFO     | archon.completions.archon:initialize_layer:79 - Initialized layer with 1 components
2024-10-22 04:41:06.308 | INFO     | archon.completions.archon:initialize_layer:79 - Initialized layer with 1 components


Model initialized: Qwen/Qwen2-72B-Instruct
Model initialized: Qwen/Qwen2-72B-Instruct
Archon initialized with 2 layers.
(Verse 1)
Gonna make a cake, it's gonna be divine,
Measure out your flour, 2 cups just right,
Baking powder 2 teaspoons, pinch of salt so fine,
Set them all aside, in a bowl they’ll align.

(Chorus)
Cream the butter, add the sugar, mix it high and low,
Eggs one by one, vanilla too, let the cake batter flow,
Alternate flour and milk, don’t overmix, you know,
Pour and smooth it out, to the oven let it go.

(Verse 2)
Preheat that oven, to 350 degrees,
Prepare your pans, grease and line with ease,
Two 9-inch rounds, ready for the feast,
Parchment paper’s key, for cakes that won’t cease.

(Chorus)
Cream the butter, add the sugar, mix it high and low,
Eggs one by one, vanilla too, let the cake batter flow,
Alternate flour and milk, don’t overmix, you know,
Pour and smooth it out, to the oven let it go.

(Bridge)
Room temp eggs and milk, butter too,
For a blend so smooth, yo